# Verificando se a "nova" base de dados está ok

O notebook disponível em `src/tratando_base_dados.ipynb` modifica a base de dados original BID, adicionando um novo arquivo para cada imagem, que é um arquivo JSON contendo informações relevantes do documento, como nome, data de nascimento, RG e CPF.

Além disso, também é criado um arquivo `.csv` com cada linha contendo informações sobre cada documento, como os arquivos correspondentes a eles, os seus dados importantes e o seu ID.

## Importações

In [1]:
import pandas as pd

## Constantes

In [2]:
DATASET_FOLDER_PATH = '../../RG-Dataset/'
DATASET_CSV_PATH = f'{DATASET_FOLDER_PATH}/dataset.csv'

## Lendo o arquivo "dataset.csv"

In [4]:
dataset = pd.read_csv(DATASET_CSV_PATH, sep=';')
dataset

,id,image_path,ocr_path,segmentation_path,info_path,cpf,rg,birthdate,name
0,000111111,files/000111111_in.jpg,files/000111111_gt_ocr.txt,files/000111111_gt_segmentation.jpg,files/000111111_info.json,354.205.532-87,08.096.661-5,02/01/1963,Rebelo Ronei Nakamurakare
1,000230000,files/000230000_in.jpg,files/000230000_gt_ocr.txt,files/000230000_gt_segmentation.jpg,files/000230000_info.json,188.354.397-52,29.227.222-4,05/05/1984,Kohatsu Liberatti Ivan
2,000233025,files/000233025_in.jpg,files/000233025_gt_ocr.txt,files/000233025_gt_segmentation.jpg,files/000233025_info.json,370.678.495-51,73.377.624-3,16/02/1976,Chicaro Okubaro Salvo
3,000233331,files/000233331_in.jpg,files/000233331_gt_ocr.txt,files/000233331_gt_segmentation.jpg,files/000233331_info.json,624.476.345-95,84.941.430-1,20/11/2008,Hochun Cerdeira Crema
4,000250000,files/000250000_in.jpg,files/000250000_gt_ocr.txt,files/000250000_gt_segmentation.jpg,files/000250000_info.json,NaN,48.753.318-5,20/07/1978,Scrignoli Petenusci Rombach
...,...,...,...,...,...,...,...,...,...
912,00026860_,files/00026860_in.jpg,files/00026860_gt_ocr.txt,files/00026860_gt_segmentation.jpg,files/00026860__info.json,291.380.124-21,80.260.201-0,27/11/2008,Palfi Morigaki Vasters
913,00026861_,files/00026861_in.jpg,files/00026861_gt_ocr.txt,files/00026861_gt_segmentation.jpg,files/00026861__info.json,184.843.419-76,28.321.208-1,17/02/1978,Hime Sylvestre Eiryo
914,00026862_,files/00026862_in.jpg,files/00026862_gt_ocr.txt,files/00026862_gt_segmentation.jpg,files/00026862__info.json,973.352.343-79,27.205.367-3,25/10/1958,Ciuffi Prandini Restum
915,00026863_,files/00026863_in.jpg,files/00026863_gt_ocr.txt,files/00026863_gt_segmentation.jpg,files/00026863__info.json,579.129.745-98,04.355.100-2,07/05/2002,Avelas Wilkens Lacaz


In [ ]:
docs_count = dataset.shape[0]
print(f'Existem {docs_count} documentos no dataset')

In [ ]:
nan_docs_count = dataset.isna()
print(f'Existem {nan_docs_count} documentos com dados nulos')

In [ ]:
not_nan_docs_count = docs_count - nan_docs_count
print(f'Logo, temos {not_nan_docs_count} documentos sem nenhum dado nulo')

## Código para vizualizar informações do dataset

In [ ]:
DOC_NUMBER = 0

current_doc = dataset.iloc[DOC_NUMBER]

## Corrigindo problema com underline no ID

In [ ]:
for i, row in dataset.iterrows():
    id = row['id']
    if id[-1] == '_':
        # Mudar nomes dos arquivos JSON
        pass

# Mudar os ids vetorizadamente
# E depois salvar o arquivo